# Feature Selection & Cleaning
The mentor suggested to run a desicion tree model to select the relevant lab tests.

## Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sb
import warnings

%matplotlib inline

color = sb.color_palette("Set2")[0]
palette = sb.color_palette("Set2")

In [2]:
data11 = pd.read_csv('data/COVID_DSL_01.CSV', sep='|', encoding = 'ISO-8859-1')
data11.head()

,EDAD,SEX,IDINGRESO,F_INGRESO_ING,F_ALTA_ING,MOTIVO_ALTA_ING,DIAGNOSTICO_ING,F_INGRESO_URG,HORA_URG,DIAG_URG,...,GLU_ULTIMA_URG,DESTINO_URG,IDCDSL,F_ING_ANT,DIAG_ANT,RESPIRADOR,F_UCI_IN,F_UCI_OUT,UCI_DAYS,UCI_N_ING
0,55,MALE,508,2020-04-05 00:00:00,2020-04-15 00:00:00,Domicilio,COVID CONFIRMADO,05/04/2020,22:28,DIFICULTAD RESPIRATORIA,...,0,Ingreso,508,NaN,NaN,SI,NaN,NaN,NaN,NaN
1,54,MALE,509,2020-04-05 00:00:00,2020-04-14 00:00:00,Domicilio,COVID CONFIRMADO,05/04/2020,22:32,DIFICULTAD RESPIRATORIA,...,0,Ingreso,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,73,MALE,510,2020-04-05 00:00:00,2020-04-20 00:00:00,Domicilio,COVID CONFIRMADO,05/04/2020,22:35,DIFICULTAD RESPIRATORIA,...,0,Ingreso,510,NaN,NaN,SI,NaN,NaN,NaN,NaN
3,90,FEMALE,511,2020-04-06 00:00:00,2020-04-07 00:00:00,Domicilio,COVID SOSPECHA,05/04/2020,23:46,DIFICULTAD RESPIRATORIA,...,0,Ingreso,511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,70,FEMALE,512,2020-04-06 00:00:00,2020-04-28 00:00:00,Traslado al Hospital,COVID CONFIRMADO,06/04/2020,00:57,DIFICULTAD RESPIRATORIA,...,0,Ingreso,512,NaN,NaN,SI,NaN,NaN,NaN,NaN


In [3]:
data11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4479 entries, 0 to 4478
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   EDAD                         4479 non-null   int64  
 1   SEX                          4479 non-null   object 
 2   IDINGRESO                    4479 non-null   int64  
 3   F_INGRESO_ING                4479 non-null   object 
 4   F_ALTA_ING                   4479 non-null   object 
 5   MOTIVO_ALTA_ING              4320 non-null   object 
 6   DIAGNOSTICO_ING              4479 non-null   object 
 7   F_INGRESO_URG                4287 non-null   object 
 8   HORA_URG                     4287 non-null   object 
 9   DIAG_URG                     4287 non-null   object 
 10  ESPECIALIDAD_URGENCIA        4287 non-null   object 
 11  HORA_CONSTANTES_PRIMERA_URG  3673 non-null   object 
 12  TA_MAX_PRIMERA_URG           4479 non-null   int64  
 13  TA_MIN_PRIMERA_URG

In [4]:
# Lab table
warnings.simplefilter('ignore')

data16 = pd.read_csv('data/COVID_DSL_06.CSV', sep=',', encoding = 'Utf-16', error_bad_lines = False)
data16.head()

b'Skipping line 100: expected 8 fields, saw 14\nSkipping line 2416: expected 8 fields, saw 9\nSkipping line 2542: expected 8 fields, saw 9\nSkipping line 2794: expected 8 fields, saw 9\nSkipping line 3293: expected 8 fields, saw 10\nSkipping line 8320: expected 8 fields, saw 14\nSkipping line 8348: expected 8 fields, saw 14\nSkipping line 8403: expected 8 fields, saw 14\nSkipping line 8901: expected 8 fields, saw 11\nSkipping line 9342: expected 8 fields, saw 9\nSkipping line 12227: expected 8 fields, saw 9\nSkipping line 13511: expected 8 fields, saw 9\nSkipping line 15227: expected 8 fields, saw 9\nSkipping line 15323: expected 8 fields, saw 9\nSkipping line 15888: expected 8 fields, saw 9\nSkipping line 16078: expected 8 fields, saw 10\nSkipping line 16966: expected 8 fields, saw 9\nSkipping line 17587: expected 8 fields, saw 9\nSkipping line 17898: expected 8 fields, saw 13\nSkipping line 18840: expected 8 fields, saw 13\nSkipping line 22366: expected 8 fields, saw 9\nSkipping line

,IDINGRESO,LAB_NUMBER,LAB_DATE,TIME_LAB,ITEM_LAB,VAL_RESULT,UD_RESULT,REF_VALUES
0,1,I-1,09-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,Se detecta,NaN,NaN
1,22,I-1,18-12-2020,00:00,SO2C -- GASOMETRIA ARTERIAL,98.0,%,(94.0 - 100.0)
2,24,I-2,21-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: Exudado...,No se detecta,NaN,NaN
3,32,I-1,18-12-2020,00:00,BT -- BILIRRUBINA TOTAL,0.42,mg/dL,<1.2
4,32,I-1,18-12-2020,00:00,HCTO -- SISTEMATICO DE SANGRE,40.9,%,(40.0 - 52.0)


In [34]:
data16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219091 entries, 0 to 219090
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   IDINGRESO   219091 non-null  int64 
 1   LAB_NUMBER  219091 non-null  object
 2   LAB_DATE    219091 non-null  object
 3   TIME_LAB    219091 non-null  object
 4   ITEM_LAB    219091 non-null  object
 5   VAL_RESULT  217407 non-null  object
 6   UD_RESULT   202137 non-null  object
 7   REF_VALUES  210247 non-null  object
dtypes: int64(1), object(7)
memory usage: 13.4+ MB


## Cleaning
Table 1 = Main
- Remove the patient who escaped from the hospital and reindex
- Create categorical variabel of "Mortivo_alta" column: "Recovery", "Recovery in Progress", "Death"

Table 6 = Labworks
- Pivot pressumed important lab tests

Combination table
- Combine the main table & the pivoted labworks

In [5]:
# Always clean of copied df
main = data11.copy()
labworks = data16.copy()

In [6]:
# Remove the one patient who escaped from the hospital from main
drop_idx = main[main.MOTIVO_ALTA_ING == 'Fuga'].index[0]
main.drop(index = drop_idx, axis=0, inplace=True)

In [7]:
# Test removal
sum(main[main.index == drop_idx].index == drop_idx)

0

In [8]:
# Reset index main
main.reset_index(inplace=True)

In [9]:
# Pivot labworks
tests = ['PLAQ -- SISTEMATICO DE SANGRE', 'PCR -- PROTEINA C REACTIVA', 'CREA -- CREATININA', 'LIN -- SISTEMATICO DE SANGRE', 'DD -- DIMERO D']
labworks_ed = labworks[labworks.ITEM_LAB.isin(tests)]
labworks_ed

,IDINGRESO,LAB_NUMBER,LAB_DATE,TIME_LAB,ITEM_LAB,VAL_RESULT,UD_RESULT,REF_VALUES
25,101,I-1,21-12-2020,00:00,PLAQ -- SISTEMATICO DE SANGRE,290.0,x10e3/µL,(150.0 - 450.0)
37,157,I-2,29-01-2021,00:00,PCR -- PROTEINA C REACTIVA,9.52,mg/L,<5
38,157,I-2,29-01-2021,00:00,PLAQ -- SISTEMATICO DE SANGRE,257.0,x10e3/µL,(150.0 - 450.0)
40,162,I-4,11-01-2021,00:00,CREA -- CREATININA,0.56,mg/dL,(0.7 - 1.2)
43,162,I-4,11-01-2021,00:00,LIN -- SISTEMATICO DE SANGRE,0.73,x10e3/µL,(1.2 - 3.4)
...,...,...,...,...,...,...,...,...
219027,4248,I-3,05-11-2020,00:00,PCR -- PROTEINA C REACTIVA,30.52,mg/L,<5
219028,4248,I-3,05-11-2020,00:00,PLAQ -- SISTEMATICO DE SANGRE,218.0,x10e3/µL,(150.0 - 450.0)
219042,4285,I-3,08-11-2020,00:00,DD -- DIMERO D,1125.0,ng/mL,<500
219067,4373,I-1,25-11-2020,00:00,LIN -- SISTEMATICO DE SANGRE,2.69,x10e3/µL,(1.2 - 3.4)


### Feature Reduction

In [10]:
main.head()

,index,EDAD,SEX,IDINGRESO,F_INGRESO_ING,F_ALTA_ING,MOTIVO_ALTA_ING,DIAGNOSTICO_ING,F_INGRESO_URG,HORA_URG,...,GLU_ULTIMA_URG,DESTINO_URG,IDCDSL,F_ING_ANT,DIAG_ANT,RESPIRADOR,F_UCI_IN,F_UCI_OUT,UCI_DAYS,UCI_N_ING
0,0,55,MALE,508,2020-04-05 00:00:00,2020-04-15 00:00:00,Domicilio,COVID CONFIRMADO,05/04/2020,22:28,...,0,Ingreso,508,NaN,NaN,SI,NaN,NaN,NaN,NaN
1,1,54,MALE,509,2020-04-05 00:00:00,2020-04-14 00:00:00,Domicilio,COVID CONFIRMADO,05/04/2020,22:32,...,0,Ingreso,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,73,MALE,510,2020-04-05 00:00:00,2020-04-20 00:00:00,Domicilio,COVID CONFIRMADO,05/04/2020,22:35,...,0,Ingreso,510,NaN,NaN,SI,NaN,NaN,NaN,NaN
3,3,90,FEMALE,511,2020-04-06 00:00:00,2020-04-07 00:00:00,Domicilio,COVID SOSPECHA,05/04/2020,23:46,...,0,Ingreso,511,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,70,FEMALE,512,2020-04-06 00:00:00,2020-04-28 00:00:00,Traslado al Hospital,COVID CONFIRMADO,06/04/2020,00:57,...,0,Ingreso,512,NaN,NaN,SI,NaN,NaN,NaN,NaN


In [11]:
# Main
keep_list = ['EDAD', 'SEX', 'IDINGRESO',
             'TA_MAX_PRIMERA_URG', 'TA_MIN_PRIMERA_URG',
             'TEMP_PRIMERA_URG', 'FC_PRIMERA_URG', 'SAT_02_PRIMERA_URG',
             'GLU_PRIMERA_URG', 'MOTIVO_ALTA_ING', 'IDCDSL', 
             'UCI_DAYS']
main_ed = main[keep_list] # keep the listed columns

In [12]:
# Change dtypes
main_ed.IDINGRESO = main_ed.IDINGRESO.astype('str')

In [13]:
# Replace commas with dots
for idx, val in enumerate(main_ed.TEMP_PRIMERA_URG):
    #print(idx, val)
    #print(labworks_ed.ITEM_LAB.iloc[idx])
    main_ed.TEMP_PRIMERA_URG.iloc[idx] = main_ed.TEMP_PRIMERA_URG.iloc[idx].replace(',', '.')
    #print(labworks_ed.ITEM_LAB.iloc[idx])
    
main_ed.TEMP_PRIMERA_URG

# Change dtype from object to float
main_ed.TEMP_PRIMERA_URG = main_ed.TEMP_PRIMERA_URG.astype(float)

In [14]:
# Labworks_ed
print(f'Shape before: {labworks.shape}')

drop_list = ['LAB_NUMBER', 'LAB_DATE', 'TIME_LAB', 'UD_RESULT', 'REF_VALUES']
labworks_ed.drop(drop_list, axis=1, inplace=True)
labworks_ed.reset_index(inplace=True)
print(f'Shape after: {labworks_ed.shape}')

Shape before: (219091, 8)
Shape after: (20232, 4)


In [15]:
lab_tests = list(labworks_ed.ITEM_LAB.unique())
lab_tests

['PLAQ -- SISTEMATICO DE SANGRE',
 'PCR -- PROTEINA C REACTIVA',
 'CREA -- CREATININA',
 'LIN -- SISTEMATICO DE SANGRE',
 'DD -- DIMERO D']

In [16]:
# Change ID to str = Unique
labworks_ed.IDINGRESO = labworks_ed.IDINGRESO.astype('str')

# Change VAL_RESULT to float
labworks_ed.VAL_RESULT = pd.to_numeric(labworks_ed.VAL_RESULT, errors='coerce')

In [17]:
# Remove blank spaces before pivoting
for idx, val in enumerate(labworks_ed.ITEM_LAB):
    #print(idx, val)
    #print(labworks_ed.ITEM_LAB.iloc[idx])
    labworks_ed.ITEM_LAB.iloc[idx] = labworks_ed.ITEM_LAB.iloc[idx].replace(' ', '')
    #print(labworks_ed.ITEM_LAB.iloc[idx])
    
labworks_ed.ITEM_LAB

0        PLAQ--SISTEMATICODESANGRE
1           PCR--PROTEINACREACTIVA
2        PLAQ--SISTEMATICODESANGRE
3                 CREA--CREATININA
4         LIN--SISTEMATICODESANGRE
                   ...            
20227       PCR--PROTEINACREACTIVA
20228    PLAQ--SISTEMATICODESANGRE
20229                  DD--DIMEROD
20230     LIN--SISTEMATICODESANGRE
20231             CREA--CREATININA
Name: ITEM_LAB, Length: 20232, dtype: object

In [18]:
# Pivot labworks
labworks_piv = labworks_ed.pivot_table(index = 'IDINGRESO', columns = 'ITEM_LAB', values = 'VAL_RESULT')
labworks_piv

ITEM_LAB,CREA--CREATININA,DD--DIMEROD,LIN--SISTEMATICODESANGRE,PCR--PROTEINACREACTIVA,PLAQ--SISTEMATICODESANGRE
IDINGRESO,,,,,
1001,0.676667,600.714286,2.258571,11.286667,363.714286
1002,1.890000,2070.000000,0.485000,99.690000,216.500000
1003,0.760000,1691.000000,2.215000,1.305000,267.500000
1004,1.260000,366.000000,1.125000,142.740000,469.000000
1007,0.890000,412.000000,1.230000,13.575000,342.000000
...,...,...,...,...,...
994,0.670000,709.333333,1.513333,71.646667,233.333333
995,1.350000,8647.500000,1.085000,290.085000,260.500000
996,0.918000,257.500000,1.510000,13.995000,210.000000


### Feature Engineering

In [19]:
# Insert new categorical column
main_ed.insert(loc = 6, column = 'PATIENT_OUTCOME', value = 'null')

In [20]:
for idx, val in enumerate(main.MOTIVO_ALTA_ING):
    #print(idx, val)
    
    if val == 'Domicilio': # Domicilio
        main_ed.PATIENT_OUTCOME[idx] = 'RECOVERED'
    
    elif val == 'Fallecimiento':
        main_ed.PATIENT_OUTCOME[idx] = 'DEATH'
        
    else:
        main_ed.PATIENT_OUTCOME[idx] = 'RECOVERY_IN_PROGRESS'

main_ed.PATIENT_OUTCOME

0                  RECOVERED
1                  RECOVERED
2                  RECOVERED
3                  RECOVERED
4       RECOVERY_IN_PROGRESS
                ...         
4473    RECOVERY_IN_PROGRESS
4474               RECOVERED
4475               RECOVERED
4476               RECOVERED
4477               RECOVERED
Name: PATIENT_OUTCOME, Length: 4478, dtype: object

In [21]:
main_ed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4478 entries, 0 to 4477
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EDAD                4478 non-null   int64  
 1   SEX                 4478 non-null   object 
 2   IDINGRESO           4478 non-null   object 
 3   TA_MAX_PRIMERA_URG  4478 non-null   int64  
 4   TA_MIN_PRIMERA_URG  4478 non-null   int64  
 5   TEMP_PRIMERA_URG    4478 non-null   float64
 6   PATIENT_OUTCOME     4478 non-null   object 
 7   FC_PRIMERA_URG      4478 non-null   int64  
 8   SAT_02_PRIMERA_URG  4478 non-null   int64  
 9   GLU_PRIMERA_URG     4478 non-null   int64  
 10  MOTIVO_ALTA_ING     4319 non-null   object 
 11  IDCDSL              4478 non-null   int64  
 12  UCI_DAYS            398 non-null    float64
dtypes: float64(2), int64(7), object(4)
memory usage: 454.9+ KB


In [22]:
# Remove the last columns
main_ed = main_ed.drop('MOTIVO_ALTA_ING', axis = 1)

In [23]:
main_ed.columns

Index(['EDAD', 'SEX', 'IDINGRESO', 'TA_MAX_PRIMERA_URG', 'TA_MIN_PRIMERA_URG',
       'TEMP_PRIMERA_URG', 'PATIENT_OUTCOME', 'FC_PRIMERA_URG',
       'SAT_02_PRIMERA_URG', 'GLU_PRIMERA_URG', 'IDCDSL', 'UCI_DAYS'],
      dtype='object')

## Combine Tables

In [24]:
main_ed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4478 entries, 0 to 4477
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EDAD                4478 non-null   int64  
 1   SEX                 4478 non-null   object 
 2   IDINGRESO           4478 non-null   object 
 3   TA_MAX_PRIMERA_URG  4478 non-null   int64  
 4   TA_MIN_PRIMERA_URG  4478 non-null   int64  
 5   TEMP_PRIMERA_URG    4478 non-null   float64
 6   PATIENT_OUTCOME     4478 non-null   object 
 7   FC_PRIMERA_URG      4478 non-null   int64  
 8   SAT_02_PRIMERA_URG  4478 non-null   int64  
 9   GLU_PRIMERA_URG     4478 non-null   int64  
 10  IDCDSL              4478 non-null   int64  
 11  UCI_DAYS            398 non-null    float64
dtypes: float64(2), int64(7), object(3)
memory usage: 419.9+ KB


In [25]:
labworks_piv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 934 entries, 1001 to 998
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CREA--CREATININA           924 non-null    float64
 1   DD--DIMEROD                883 non-null    float64
 2   LIN--SISTEMATICODESANGRE   928 non-null    float64
 3   PCR--PROTEINACREACTIVA     898 non-null    float64
 4   PLAQ--SISTEMATICODESANGRE  928 non-null    float64
dtypes: float64(5)
memory usage: 43.8+ KB


In [26]:
data = main_ed.merge(labworks_piv, on = 'IDINGRESO')

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 933
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   EDAD                       934 non-null    int64  
 1   SEX                        934 non-null    object 
 2   IDINGRESO                  934 non-null    object 
 3   TA_MAX_PRIMERA_URG         934 non-null    int64  
 4   TA_MIN_PRIMERA_URG         934 non-null    int64  
 5   TEMP_PRIMERA_URG           934 non-null    float64
 6   PATIENT_OUTCOME            934 non-null    object 
 7   FC_PRIMERA_URG             934 non-null    int64  
 8   SAT_02_PRIMERA_URG         934 non-null    int64  
 9   GLU_PRIMERA_URG            934 non-null    int64  
 10  IDCDSL                     934 non-null    int64  
 11  UCI_DAYS                   119 non-null    float64
 12  CREA--CREATININA           924 non-null    float64
 13  DD--DIMEROD                883 non-null    float64

In [28]:
data.head()

,EDAD,SEX,IDINGRESO,TA_MAX_PRIMERA_URG,TA_MIN_PRIMERA_URG,TEMP_PRIMERA_URG,PATIENT_OUTCOME,FC_PRIMERA_URG,SAT_02_PRIMERA_URG,GLU_PRIMERA_URG,IDCDSL,UCI_DAYS,CREA--CREATININA,DD--DIMEROD,LIN--SISTEMATICODESANGRE,PCR--PROTEINACREACTIVA,PLAQ--SISTEMATICODESANGRE
0,55,MALE,508,0,0,0.0,RECOVERED,0,0,0,508,NaN,0.705,399.000000,2.290,6.080000,472.000000
1,54,MALE,509,0,0,0.0,RECOVERED,0,0,0,509,NaN,0.800,398.000000,2.355,13.310000,657.000000
2,73,MALE,510,0,0,0.0,RECOVERED,0,0,0,510,NaN,1.056,4396.750000,1.482,3.760000,306.000000
3,70,FEMALE,512,0,0,0.0,RECOVERY_IN_PROGRESS,0,0,0,512,NaN,0.670,497.333333,1.395,21.986667,278.166667
4,61,FEMALE,513,0,0,0.0,RECOVERED,0,0,0,513,NaN,0.670,470.000000,1.010,7.370000,318.000000


### One-Hot Encoding

In [29]:
# Sex
data[['FEMALE', 'MALE']] = pd.get_dummies(data.SEX)

In [30]:
sorted(list(data.PATIENT_OUTCOME.unique()))

['DEATH', 'RECOVERED', 'RECOVERY_IN_PROGRESS']

In [31]:
# PATIENT_OUTCOME
sorted_list = sorted(list(data.PATIENT_OUTCOME.unique()))
data[sorted_list]=pd.get_dummies(data.PATIENT_OUTCOME)

In [32]:
# Remove the orgiginal columns
data = data.drop(['SEX', 'PATIENT_OUTCOME'], axis=1)

## Save Cleaned Data


In [33]:
# Saving data batch 1

path = 'data/'
file_name = 'cleaned_data_v1-1.csv'
file_path = path + file_name
data.to_csv(file_path)